In [ ]:
# https://workspace-preview.neo4j.io/workspace/query

In [129]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph
import pandas as pd

In [130]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('AURA_INSTANCENAME') # don't need this I guess

In [131]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

In [71]:
cypher = """
CREATE (alan:Person {name:"Alan"})
RETURN alan
"""
kg.query(cypher)

[{'alan': {'name': 'Alan'}}]

In [97]:
cypher = """
Match (n)
RETURN count(n)
"""
kg.query(cypher)

[{'count(n)': 0}]

In [31]:
def addPerson(personname):
    cypher = """
        MATCH (p:Person {name: $personname})
        RETURN count(p)
    """
    result = kg.query(cypher, {'personname':personname})
    if result[0]['count(p)'] != 0:
        return

    cypher = """
        CREATE (p:Person {name: $personname})
        RETURN p
    """
    kg.query(cypher, {'personname':personname})

In [35]:
# addPerson("ALAN")

In [126]:
def delete_graph():
    cypher = """
        MATCH (n)
        DETACH DELETE n
    """
    kg.query(cypher)
delete_graph()

In [125]:
df = pd.read_csv('buyer_seller_hierarchical_data_intersections.csv')
df

,Seller,Buyer
0,VUZXR,TCCTN
1,TCCTN,GPAPJ
2,GPAPJ,NXRVO
3,NXRVO,WSDNZ
4,WSDNZ,JQZFM
...,...,...
95,BAJWR,EOWNT
96,EOWNT,VCRHZ
97,VCRHZ,KMDTS
98,KMDTS,IEBUQ


In [99]:
def addCompany(company_name):
    cypher = """
        MATCH (c:Company {name: $company_name})
        RETURN count(c)
    """
    result = kg.query(cypher, {'company_name':company_name})
    if result[0]['count(c)'] != 0:
        return

    cypher = """
        CREATE (c:Company {name: $company_name})
        RETURN c
    """
    kg.query(cypher, {'company_name':company_name})

In [121]:
def addRealationship(company_name1, company_name2):
    # probably put a check to see if it already exists
    cypher = """
        MATCH (c1:Company {name: $company_name1}), (c2:Company {name: $company_name2})
        CREATE (c1)-[r:SELLS_TO]->(c2)
        RETURN c1, c2
    """
    kg.query(cypher, {'company_name1':company_name1, 'company_name2':company_name2})

In [115]:
cypher = """
    MATCH (c1:Company {name: 'QBBQI'}), (c2:Company {name: 'XRIMJ'})
    CREATE (c1)-[r:BUYS_FROM]->(c2)
    RETURN r
"""
# kg.query(cypher)

[{'r': ({}, 'BUYS_FROM', {})}]

In [109]:
cypher = """
    MATCH (c:Company {name:'QBBQI'})
    RETURN count(c)
"""
kg.query(cypher)

cypher = """
    MATCH (c:Company {name:'XRIMJ'})
    RETURN count(c)
"""
kg.query(cypher)

# addRealationship('QBBQI', 'XRIMJ')

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Compnay)} {position: line: 2, column: 11, offset: 19} for query: '\n        MATCH (c1:Compnay {name: $company_name1}), (c2:Company {name: $company_name2})\n        RETURN c1, c2\n    '


In [127]:
for index, row in df.iterrows():
    addCompany(row['Seller'])
    addCompany(row['Buyer'])
    addRealationship(row['Seller'], row['Buyer'])

In [110]:
cypher = """
    MATCH (c:Company {name:'QBBQI'})
    RETURN count(c)
"""
kg.query(cypher)

[{'count(c)': 1}]

In [128]:
# find chain of realtions ending at 'JOBQV'
cypher = """
    MATCH c = (c:Company)-[:SELLS_TO*]->(c:Company {name: 'JOBQV'})
    RETURN c
"""
kg.query(cypher)

Failed to read from defunct connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information